In [1]:
import pandas as pd
import numpy as np
import config
import tensorflow as tf

import matplotlib.pyplot as plt
import seaborn as sns

import keras
from keras.models import Model, load_model, save_model
from keras.applications import DenseNet201, densenet
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Lambda

from pathlib import Path

/home/mohsin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def build_model():
    bb_model = DenseNet201(include_top=False, pooling='avg', input_shape=(224,224,3))
    l2norm = Lambda(lambda x: tf.nn.l2_normalize(x, axis=1))(bb_model.output)
    l2_model = Model(inputs=[bb_model.input], outputs=l2norm)
    l2_model.summary()
    return l2_model

In [3]:
batch_size = 32
datagen = ImageDataGenerator(preprocessing_function=densenet.preprocess_input)
generator = datagen.flow_from_directory( "data/",
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

Found 3026 images belonging to 1 classes.


In [4]:
model = build_model()
num_samples = generator.samples
num_batches = np.ceil(num_samples / batch_size)
embeddings = model.predict_generator(generator, num_batches, verbose=1)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

95/95 [==============================] - 18s 188ms/step


In [5]:
embeddings.shape

(3026, 1920)

In [6]:
from keras.layers import Dense, Dropout, Input

In [7]:
files = generator.filenames

In [21]:
def build_ae_model(inp_dim=1920, hidden_dim=128):
    input_img = Input(shape=(inp_dim,))
    drop = Dropout(0.4)(input_img)
    h1 = Dense(256, activation='relu', name='h1')(drop)
    encoded = Dense(hidden_dim, activation='relu', name='encoder')(h1)
    d1 = Dense(256, activation='relu', name='h2')(encoded)
    decoded = Dense(inp_dim, activation='relu', name='decoder')(d1)
    autoencoder = Model(input_img, decoded)
    autoencoder.compile(optimizer='adam', loss='mean_squared_error')
    return autoencoder

In [22]:
def build_encoder(ae_model):
    model = Model(ae_model.input, ae_model.get_layer('encoder').output)
    return model

In [23]:
ae_model = build_ae_model(hidden_dim=32)
ae_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 1920)              0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 1920)              0         
_________________________________________________________________
h1 (Dense)                   (None, 256)               491776    
_________________________________________________________________
encoder (Dense)              (None, 32)                8224      
_________________________________________________________________
h2 (Dense)                   (None, 256)               8448      
_________________________________________________________________
decoder (Dense)              (None, 1920)              493440    
Total params: 1,001,888
Trainable params: 1,001,888
Non-trainable params: 0
_________________________________________________________________


In [24]:
ae_model.fit(embeddings, embeddings, batch_size=128, epochs=300, shuffle=True)

Epoch 1/300
3026/3026 [==============================] - 3s 901us/step - loss: 3.5349e-04
Epoch 2/300
3026/3026 [==============================] - 0s 38us/step - loss: 2.3919e-04
Epoch 3/300
3026/3026 [==============================] - 0s 39us/step - loss: 2.0346e-04
Epoch 4/300
3026/3026 [==============================] - 0s 40us/step - loss: 1.9089e-04
Epoch 5/300
3026/3026 [==============================] - 0s 39us/step - loss: 1.7815e-04
Epoch 6/300
3026/3026 [==============================] - 0s 40us/step - loss: 1.7029e-04
Epoch 7/300
3026/3026 [==============================] - 0s 40us/step - loss: 1.6339e-04
Epoch 8/300
3026/3026 [==============================] - 0s 39us/step - loss: 1.5714e-04
Epoch 9/300
3026/3026 [==============================] - 0s 40us/step - loss: 1.5135e-04
Epoch 10/300
3026/3026 [==============================] - 0s 39us/step - loss: 1.4749e-04
Epoch 11/300
3026/3026 [==============================] - 0s 39us/step - loss: 1.4424e-04
Epoch 12/300
3026/

3026/3026 [==============================] - 0s 39us/step - loss: 9.3297e-05
Epoch 93/300
3026/3026 [==============================] - 0s 39us/step - loss: 9.3201e-05
Epoch 94/300
3026/3026 [==============================] - 0s 40us/step - loss: 9.2937e-05
Epoch 95/300
3026/3026 [==============================] - 0s 39us/step - loss: 9.2941e-05
Epoch 96/300
3026/3026 [==============================] - 0s 40us/step - loss: 9.2957e-05
Epoch 97/300
3026/3026 [==============================] - 0s 38us/step - loss: 9.2822e-05
Epoch 98/300
3026/3026 [==============================] - 0s 40us/step - loss: 9.2732e-05
Epoch 99/300
3026/3026 [==============================] - 0s 39us/step - loss: 9.2595e-05
Epoch 100/300
3026/3026 [==============================] - 0s 39us/step - loss: 9.2588e-05
Epoch 101/300
3026/3026 [==============================] - 0s 41us/step - loss: 9.2372e-05
Epoch 102/300
3026/3026 [==============================] - 0s 37us/step - loss: 9.2292e-05
Epoch 103/300
3026/3

3026/3026 [==============================] - 0s 40us/step - loss: 8.5298e-05
Epoch 183/300
3026/3026 [==============================] - 0s 38us/step - loss: 8.5344e-05
Epoch 184/300
3026/3026 [==============================] - 0s 38us/step - loss: 8.5237e-05
Epoch 185/300
3026/3026 [==============================] - 0s 37us/step - loss: 8.5104e-05
Epoch 186/300
3026/3026 [==============================] - 0s 37us/step - loss: 8.5135e-05
Epoch 187/300
3026/3026 [==============================] - 0s 40us/step - loss: 8.4991e-05
Epoch 188/300
3026/3026 [==============================] - 0s 38us/step - loss: 8.4894e-05
Epoch 189/300
3026/3026 [==============================] - 0s 38us/step - loss: 8.4859e-05
Epoch 190/300
3026/3026 [==============================] - 0s 39us/step - loss: 8.4961e-05
Epoch 191/300
3026/3026 [==============================] - 0s 37us/step - loss: 8.5019e-05
Epoch 192/300
3026/3026 [==============================] - 0s 38us/step - loss: 8.4873e-05
Epoch 193/300

3026/3026 [==============================] - 0s 39us/step - loss: 8.1766e-05
Epoch 273/300
3026/3026 [==============================] - 0s 40us/step - loss: 8.1952e-05
Epoch 274/300
3026/3026 [==============================] - 0s 40us/step - loss: 8.1757e-05
Epoch 275/300
3026/3026 [==============================] - 0s 40us/step - loss: 8.1853e-05
Epoch 276/300
3026/3026 [==============================] - 0s 40us/step - loss: 8.1722e-05
Epoch 277/300
3026/3026 [==============================] - 0s 39us/step - loss: 8.1767e-05
Epoch 278/300
3026/3026 [==============================] - 0s 41us/step - loss: 8.1610e-05
Epoch 279/300
3026/3026 [==============================] - 0s 40us/step - loss: 8.1548e-05
Epoch 280/300
3026/3026 [==============================] - 0s 39us/step - loss: 8.1710e-05
Epoch 281/300
3026/3026 [==============================] - 0s 38us/step - loss: 8.1469e-05
Epoch 282/300
3026/3026 [==============================] - 0s 39us/step - loss: 8.1753e-05
Epoch 283/300

In [25]:
ae_encoder = build_encoder(ae_model)

In [26]:
ae_embeds = ae_encoder.predict(embeddings)

In [27]:
ae_embeds.shape

(3026, 32)

In [28]:
# Check embeds

In [29]:
productids = [int(Path(f).stem) for f in files]
productids

[11139192,
 11139194,
 11139524,
 11139560,
 11139588,
 11139650,
 11141306,
 11141308,
 11141318,
 11141320,
 11141324,
 11141326,
 11141328,
 11141330,
 11141338,
 11141340,
 11141342,
 11141346,
 11141354,
 11141530,
 11141538,
 11141644,
 11144136,
 11144260,
 11145600,
 11145602,
 11145612,
 11145614,
 11145620,
 11145624,
 11145626,
 11145634,
 11145640,
 11145642,
 11145654,
 11145664,
 11145666,
 11145684,
 11145702,
 11145726,
 11145728,
 11145740,
 11145744,
 11145748,
 11145762,
 11146082,
 11146084,
 11146384,
 11146754,
 11147250,
 11147252,
 11147258,
 11147268,
 11147280,
 11147282,
 11147290,
 11147292,
 11147300,
 11147312,
 11147324,
 11147512,
 11147534,
 11147546,
 11147548,
 11147600,
 11147622,
 11148212,
 11148440,
 11148482,
 11148514,
 11148516,
 11148792,
 11148854,
 11148860,
 11148888,
 11148902,
 11148920,
 11148922,
 11148954,
 11148964,
 11148966,
 11148980,
 11148982,
 11148984,
 11149026,
 11149036,
 11149062,
 11149198,
 11149838,
 11149842,
 11149866,

In [30]:
em_dict = {}
for pid, em in zip(productids, ae_embeds):
    em_dict[pid] = em.flatten()

In [31]:
import pickle
with open( "outputs/pid_em.pkl", "wb") as f:
    pickle.dump(em_dict, f)

In [32]:
len(em_dict)

3026

In [33]:
em_dict[11139192]

array([0.        , 0.        , 0.4325103 , 0.        , 0.46405852,
       0.        , 0.38293362, 0.        , 0.24399802, 0.33453616,
       0.        , 0.        , 0.        , 0.32402   , 0.        ,
       0.        , 0.        , 0.14970759, 0.24812868, 0.5148265 ,
       0.18516803, 0.        , 0.26886356, 0.        , 0.09424812,
       0.12164172, 0.2286085 , 0.        , 0.        , 0.2314678 ,
       0.        , 0.        ], dtype=float32)